In [1]:
import numpy as np
import cv2
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
cap = cv2.VideoCapture('./train/trainGrayMini.mp4')
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

bufTemp = np.empty((frameCount, frameHeight * frameWidth), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount  and ret):
    ret, x = np.array(cap.read())
    x = x[:,:,0]
    x.shape = frameHeight*frameWidth
    bufTemp[fc] = x
    fc += 1

cap.release()

x = bufTemp[0, 0:frameHeight*frameWidth]
x.shape = (frameHeight, frameWidth)

In [3]:
y = np.loadtxt('./train/train.txt', np.dtype('float64'))
theta = np.zeros(((frameHeight*frameWidth)+1), np.dtype('float64'))
m = frameCount-1

# Alpha greater than 0.16 will overflow and not work.
# Alpha around 0.01 or lower seems to do the best.
# alpha = np.array([0.1],np.dtype('float64'))
alpha = np.array([0.00078125],np.dtype('float64'))
lmbda = np.array([0.64],np.dtype('float64'))
outerLoopCount = 10
loopCount = 20 # Was 5

difDivider = 2

alpLmbTheta = np.zeros((len(alpha), len(lmbda), ((frameHeight*frameWidth)+1)), np.dtype('float64'))
result = np.empty((len(alpha), len(lmbda), frameCount), np.dtype('float64'))
costArr = np.zeros((1),np.dtype('float64'))
beginArr = 1

stop = False
threshold = 5.0

In [4]:
# Uses bufTemp, y, m, alpha, lmbda, alpLmbTheta, loopCount, costArr
def learn():
    for idx, alp in enumerate(alpha):
        for idx2, lmbd in enumerate(lmbda):
            for i in range(loopCount):
                cost = 0
                for j in range(1,frameCount):
                    firstFrame = np.array(bufTemp[j-1], dtype='int16')
                    secondFrame = np.array(bufTemp[j], dtype='int16')
                    x = np.empty(((frameHeight*frameWidth)+1), np.dtype('float64'))
                    x[0] = 1
                    x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
                    x = np.absolute(x)
                    mean = np.mean(x)
                    x = x - mean
                    x = x/np.std(x)
                    grad = ((np.dot(x, alpLmbTheta[idx, idx2]) - y[j])*x)/m
                    grad[1:len(grad)] = grad[1:len(grad)] + ((lmbd/m)*alpLmbTheta[idx,idx2,1:len(alpLmbTheta[idx,idx2])])
                    alpLmbTheta[idx,idx2] = (alpLmbTheta[idx,idx2] - (alp*grad))
                if i % 3 == 0:
                    cost = np.zeros((1), np.dtype('float64'))
                    for idx in range(len(alpha)):
                        for idx2 in range(len(lmbda)):
                            for j in range(1,frameCount):
                                firstFrame = np.array(bufTemp[j-1], dtype='int16')
                                secondFrame = np.array(bufTemp[j], dtype='int16')
                                x = np.empty(((frameHeight*frameWidth)+1), np.dtype('float64'))
                                x[0] = 1
                                x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
                                x = np.absolute(x)
                                mean = np.mean(x)
                                x = x - mean
                                x = x/np.std(x)
                                result[idx][idx2][j-1] = np.dot(x,alpLmbTheta[idx,idx2])
                                cost = cost + (((result[idx][idx2][j-1] - y[j])**2)/2)
                            cost = cost/(2*m)
                            global costArr, beginArr
                            costArr = np.append(costArr, cost)
                            print("%d:"%(i), cost)
                            
                            if i >= 3:
                                if (costArr[-2] - costArr[-1]) <  ((costArr[beginArr] - costArr[beginArr+1])/difDivider):
                                    alpha[idx] = alp/2
#                                     costArr = np.append(costArr, 0)
                                    beginArr = len(costArr)
                                    return 1

In [5]:
def startLearn():
    learn()
    print('HELLO')
    while costArr[-1] > threshold:
        print('Hello')
        learn()
    print('hello')

In [6]:
def startLearnRange():
    learn()
    for i in range(outerLoopCount):
        learn()
        print()

In [7]:
def getResult():
    
    for idx in range(len(alpha)):
        for idx2 in range(len(lmbda)):
            for j in range(1,frameCount):
                firstFrame = np.array(bufTemp[j-1], dtype='int16')
                secondFrame = np.array(bufTemp[j], dtype='int16')
                x = np.empty(((frameHeight*frameWidth)+1), np.dtype('float64'))
                x[0] = 1
                x[1:(frameHeight*frameWidth)+1] = secondFrame - firstFrame
                x = np.absolute(x)
                mean = np.mean(x)
                x = x - mean
                x = x/np.std(x)
                result[idx][idx2][j-1] = np.dot(x,alpLmbTheta[idx,idx2])

#     result.tofile('./train/trainMini.txt',sep="\n",format="%s")

In [8]:
def getCost():
    cost = np.zeros((1), np.dtype('float64'))
    
    for idx in range(len(alpha)):
        for idx2 in range(len(lmbda)):
            for j in range(1,frameCount):
                cost = cost + (((result[idx][idx2][j-1] - y[j])**2)/2)
            cost = cost/(2*m)
            print(alpha, ":", cost)
        print()

In [9]:
alpLmbTheta[0][0] = np.loadtxt('./train/trainThetaMini.txt', np.dtype('float64'), delimiter='\n')

In [10]:
startLearnRange()
getResult()
getCost()

0: [3.58464685]
3: [3.46704639]
6: [3.40456796]
9: [3.35501498]
0: [3.19286363]
3: [3.13032703]
6: [3.1057607]

0: [3.07433478]
3: [3.04485576]
6: [3.0315291]

0: [3.02540908]
3: [3.01457529]
6: [3.00765558]
9: [3.00198281]
12: [2.99686335]

0: [2.99544572]
3: [2.99189556]
6: [2.98893154]
9: [2.98626456]
12: [2.98376883]
15: [2.98138301]
18: [2.97907414]

0: [2.97756784]
3: [2.97534838]
6: [2.97316932]
9: [2.97102462]
12: [2.96891005]
15: [2.96682245]
18: [2.96475943]

0: [2.96339679]
3: [2.9613707]
6: [2.95936487]
9: [2.95737815]
12: [2.95540957]
15: [2.95345827]
18: [2.9515235]

0: [2.95024251]
3: [2.94833386]
6: [2.94644013]
9: [2.94456079]
12: [2.94269541]
15: [2.94084355]
18: [2.93900481]

0: [2.93778611]
3: [2.93596847]
6: [2.93416305]
9: [2.93236954]
12: [2.93058766]
15: [2.92881715]

0: [2.92844666]
3: [2.92738662]
6: [2.92638236]
9: [2.92541552]
12: [2.92447471]
15: [2.9235527]
18: [2.92264474]

0: [2.92204563]
3: [2.92115455]
6: [2.92027107]
9: [2.91939398]
12: [2.91852236]
1

In [ ]:
startLearn()
getResult()
getCost()

In [ ]:
print(costArr[-1])

In [11]:
alpLmbTheta.tofile('./train/trainThetaMini2.txt',sep="\n",format="%s")

In [13]:
result.shape

(1, 1, 20398)

In [14]:
frameCount

20398

In [15]:
result.tofile('./train/trainMini.txt',sep="\n",format="%s")